In [1]:
import os
import re
import itertools as it
import pandas as pd
from platform import python_version
from collections import defaultdict
from pymongo import MongoClient

In [2]:
print(f"Python {python_version()}")
print(f"Pandas {pd.__version__}")

Python 3.6.5
Pandas 1.1.1


In [3]:
divisor_to_regex = {
    "dP": re.compile("{1,0,0,[1-9]}"),
    "NSR": re.compile("{1,0,0,[1-9][^}]"),
    "K3": re.compile("{1,0,1,20}"),
    "Wilson": re.compile("{1,[1-9]+,0,[1-9]+}"),
    "EWilson": re.compile("{1,1,0,[1-9]+}"),
    "SD1": re.compile("{1,0,1,21}"),
    "SD2": re.compile("{1,0,2,30}")
}
divisor_key_combs = list(
    it.chain(
        it.combinations(divisor_to_regex, 1),
        it.combinations(['dP', 'K3', 'Wilson'], 2),
        it.combinations(['dP', 'K3', 'EWilson'], 2),
        it.combinations(['dP', 'K3', 'Wilson'], 3),
        it.combinations(['dP', 'K3', 'EWilson'], 3)
    )
)
def divisor_func(x):
    counts = defaultdict(int)
    for divisors in x.INVOLDIVCOHOM:
        divisors = ''.join(divisors)
        for divisor_key_comb in divisor_key_combs:
            # Product of counts for each type of divisor in combination
            # (how many ways of getting that divisor combination)
            prod = 1
            for divisor_key in divisor_key_comb:
                prod *= len(divisor_to_regex[divisor_key].findall(divisors))
            counts[divisor_key_comb] += prod
    return counts

In [4]:
oplane_to_odim = {
    "O3": 3,
    "O5": 5,
    "O7": 7
}
oplane_key_combs = list(
    it.chain(
        it.combinations(oplane_to_odim, 1),
        it.combinations(oplane_to_odim, 2),
        it.combinations(oplane_to_odim, 3)
    )
)
def oplane_func(x):
    counts = defaultdict(int)
    for oplanes in x.OPLANES:
        odims = [x['ODIM'] for x in oplanes]
        for oplane_key_comb in oplane_key_combs:
            # Product of counts for each type (i.e. dimension) of O-plane in combination
            # (how many ways of getting that O-plane dimension combination)
            prod = 1
            for oplane_key in oplane_key_comb:
                prod *= odims.count(oplane_to_odim[oplane_key])
            counts[oplane_key_comb] += prod
    return counts

In [2]:
uri = os.getenv('MONGO_URI')
client = MongoClient(uri)
db = client.ToricCY

In [6]:
h11 = 4

poly_df = pd.DataFrame.from_records(db.POLY.find({"H11": h11}))
geom_df = pd.DataFrame.from_records(db.GEOM.find({"H11": h11}))
invol_df = pd.DataFrame.from_records(db.INVOL.find({"H11": h11}))

df = geom_df.merge(invol_df, how='right', on=['POLYID', 'GEOMN'])
df = poly_df.merge(df, how='right', on=['POLYID'])
df = df[df.FUNDGP == 1]

# Filter proper, geometry-wide, and definite volume form
df = df.groupby(['POLYID', 'GEOMN', 'INVOL']).filter(
    lambda x: 
        (x.SRINVOL & x.ITENSXDINVOL).all() and \
        (len(x) == x.NTRIANGS.iloc[0]) and \
        (x.VOLFORMPARITY.nunique() == 1)
)

grps = df.groupby(['POLYID', 'GEOMN', 'INVOL'])

In [7]:
# Number of geometry-wide proper involutions
len(grps)

259

In [8]:
# Number of ways of getting a divisor combination in any geometry-wide involution
pd.DataFrame.from_records(grps.apply(divisor_func).values).sum()

(dP,)                116
(NSR,)               277
(K3,)                219
(Wilson,)             25
(EWilson,)             2
(SD1,)                26
(SD2,)                18
(dP, K3)               0
(dP, Wilson)          16
(K3, Wilson)          12
(dP, EWilson)          0
(K3, EWilson)          4
(dP, K3, Wilson)       0
(dP, K3, EWilson)      0
dtype: int64

In [9]:
# Number of ways of getting a O-plane combination in any geometry-wide involution
pd.DataFrame.from_records(grps.apply(oplane_func).values).sum()

(O3,)           217
(O5,)           216
(O7,)           163
(O3, O5)          0
(O3, O7)         94
(O5, O7)          0
(O3, O5, O7)      0
dtype: int64

In [6]:
db.GEOM.find_one({"H11":2})

{'_id': ObjectId('58c2303d6ccbae3dd44b27d8'),
 'MORIMAT': '{{0,0,1,1,1,0},{1,1,0,0,0,1}}',
 'CHERN2XJ': '3*J1^2+9*J1*J2+3*J2^2',
 'KAHLERMAT': '{{1,0},{0,1}}',
 'IPOLYXJ': '3*J1^2*J2+3*J1*J2^2',
 'NTRIANGS': 1,
 'POLYID': 10,
 'CHERN2XNUMS': '{36,36}',
 'H11': 2,
 'GEOMN': 1,
 'ITENSXJ': '{{{0,3},{3,3}},{{3,3},{3,0}}}',
 'toricswisscheeseSTATS': {'MAXRSS': 12412000,
  'BSONSIZE': 0,
  'MAXVMSIZE': 313544000,
  'CPUTIME': 17},
 'toricswisscheeseMARK': True,
 'involenumSTATS': {'MAXRSS': 0,
  'BSONSIZE': 1011,
  'MAXVMSIZE': 0,
  'CPUTIME': 0.0},
 'involenumMARK': True}